# <a id='index'> 목차 </a>

<ol style="list-style-type: decimal;">
    <li><font color="black"><a href="#basic_settings"> 기본 세팅 </a></font></li>
    <ol style="list-style-type: decimal;">
        <li><a href="#library">라이브러리 </a></li>
        <li><a href="#api_key">API KEY</a></li>
    </ol>
    <li><font color="black"><a href="#download_data"> 데이터 다운로드 </a></font></li>
    <ol style="list-style-type: decimal;">
        <li><a href="#check_version"> 버전 확인 </a></li>
        <li><a href="#champ_data"> 챔피언 데이터 </a></li>
        <li><a href="#match_data"> 매치 데이터 </a></li>
        <ol style="list-style-type: decimal;">
            <li><a href="#user_id"> 티어별 유저 아이디 </a></li>
            <li><a href="#match_data_sub"> 매치 데이터 </a></li>
        </ol>
    </ol>
    <li><font color="black"> 참고 사이트 </font></li>
    <ol style="list-style-type: decimal;">
        <li><a href="https://right1203.github.io/study/2019/02/28/lol-duo-tier/"> 롤 API </a></li>
        <li><a href="https://plotly.com/python/figurewidget-app/"> Interactive Plotly </a></li>
        <li><a href="https://hogni.tistory.com/86"> Plotly Bar </a></li>
        <li><a href="https://riot-watcher.readthedocs.io/en/latest/"> LolWatchers </a></li>
    </ol>
</ol>

# <a id='basic_settings'> 1. 기본 세팅 </a> <font size=1> [목차](#index) </font>
> 1. 라이브러리
> 1. API_KEY

> ### <a id='library'> 1. 라이브러리 </a> <font size=1> [목차](#index) </font>
    - 데이터를 다운로드 하는데 필요한 라이브러리 설치

In [47]:
import os  # .env에서 API_KEY를 받아오는데 사용
import requests # LOL API로 다운로드 받을 때 사용
import json  # LOL API로 받은 json파일 읽을 때 사용
import pandas as pd  # 데이터 읽고 저장할 때 사용 
import time  # API 요청 제한 걸렸을 때 잠시 대기하는데 사용
from tqdm.notebook import tqdm  # loop 돌 때 진행정도 파악하는데 사용
from tqdm import trange
from packaging import version  # 최신 버전 가져올 때 사용
import ast  # quotes list 처리할 때 사용
import pickle  # pickle 파일 저장할 때

> ### <a id='API_KEY'> 2. API KEY </a> <font size=1> [목차](#index) </font>
> - 환경변수에 저장된 API KEY 세팅
>     - API KEY가 매일 만료되기 때문에 매 번 수정해서 사용해야 함

In [41]:
api_key = os.environ["API_KEY"]

# <a id='download_data'> 2. 데이터 다운로드 </a> <font size=1> [목차](#index) </font>
> 1. 버전확인
> 1. 챔피언 데이터
> 1. 매치 데이터
>> <ol style="list-style-type: decimal;">
    <li> 티어별 유저 아이디 </li>
    <li> 매치 데이터 </li>
   </ol>

> ### <a id='check_version'> 1. 버전 확인 </a> <font size=1> [목차](#index) </font>
> - 챔피언 데이터를 뽑을 때 사용할 롤 버전 확인

In [71]:
r = requests.get('https://ddragon.leagueoflegends.com/api/versions.json') # version data 확인
current_version = r.json()[0] # 가장 최신 버전 확인
current_version

'10.19.1'

> ### <a id='champ_data'> 2. 챔피언 데이터 </a> <font size=1> [목차](#index) </font>
> - 이후 데이터에서 챔피언 정보가 키 값으로 저장되어 있기 때문에 사용할 챔피언 마스터 데이터

In [72]:
# 현재 버전의 챔피언 데이터
api_url = f"http://ddragon.leagueoflegends.com/cdn/{current_version}/data/ko_KR/champion.json"
r = requests.get(api_url)
parsed_data = r.json() # 파싱
df_info = pd.DataFrame(parsed_data)

# df_info의 data 값들을 데이터프레임으로 변환

# 데이터의 각 행을 시리즈 형태로 변환하여 딕셔너리에 추가
dict_champ = {}
for _i, _champ in enumerate(df_info.data):
    dict_champ[_i] = pd.Series(_champ)

# 데이터 프레임 변환 후 Transpose
df_champ = pd.DataFrame(dict_champ).T

# output : 챔피언 데이터 안에도 info와 stats가 딕셔너리 형태임
# 이 데이터들을 데이터프레임으로 변환하여 각 챔피언에 대한 변수로 추가해야 한다.

# champ_df의 info, stats의 데이터를 변수로 추가
df_champ_info = pd.DataFrame(dict(df_champ['info'])).T
df_champ_stats = pd.DataFrame(dict(df_champ['stats'])).T

# 데이터 합치기
df_champ = pd.concat([df_champ, df_champ_info], axis=1)
df_champ = pd.concat([df_champ, df_champ_stats], axis=1)

# 이번 분석에서 필요없는 데이터 제거
# df_champ = df_champ.drop(['image', 'info', 'stats'], axis=1)
df_champ.to_csv("../data/df_champ.csv", index=False)
df_champ[[]].to_csv("../data/df_champ_final.csv", index=False)

> ### <a id='match_data'> 3. 매치 데이터 </a> <font size=1> [목차](#index) </font>
>> <ol>
    <li>티어별 유저 아이디</li>
    <li>매치 데이터</li>
   </ol>
> - 매치 데이터를 얻기 위해서 먼저 유저별 아이디 정보로부터 Account ID를 받아옴

>> ##### <a id='user_id'> 1. 티어별 유저 아이디 </a> <font size=1> [목차](#index) </font>
>> - 먼저, (챌린저, 그마, 마스터) 유저의 아이디를 받아온 후
>> - 게임 ID를 받아오기 위해 Account ID를 받아옴 (유저 ID와 게임 ID는 1:1 매칭)

In [47]:
# 유저 아이디 받아오기

api_url_challenger = f"https://kr.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key={api_key}"
api_url_grandmaster = f"https://kr.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5?api_key={api_key}"
api_url_master = f"https://kr.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5?api_key={api_key}"
# 그랜드마스터, 마스터 데이터 가져와서 데이터프레임으로 변환
r = requests.get(api_url_challenger)
df_league = pd.DataFrame(r.json())
r = requests.get(api_url_grandmaster)
df_league = pd.concat([df_league, pd.DataFrame(r.json())], axis=0)
r = requests.get(api_url_master)
df_league = pd.concat([df_league, pd.DataFrame(r.json())], axis=0)

# entries 데이터프레임으로 변환하여 추가
df_league.reset_index(inplace=True)
df_league_entries = pd.DataFrame(dict(df_league['entries'])).T
df_league = pd.concat([df_league, df_league_entries], axis=1)

SyntaxError: EOL while scanning string literal (<ipython-input-47-4fc44f201722>, line 3)

In [46]:
# Acount ID 받아오기

account_ids = []
for _i, _summoner_name in tqdm(enumerate(df_league.summonerName), total=df_league.shape[0]):
    try:
        api_url = f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/{_summoner_name}?api_key={api_key}"
        r = requests.get(sohwan)
        
        while r.status_code == 429:
            time.sleep(5)
            api_url = f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/{_summoner_name}?api_key={api_key}"
            r = requests.get(api_url)
            
        account_ids += [r.json()['accountId']]   
     
    except Exception as e:
        account_ids += [None]
        pass
df_league["accountId"] = account_ids
df_league = df_league.dropna().reset_index()  # accountId가 Na인 것 삭제 (ID가 바뀐 경우)
df_league.to_csv("../data/df_league.csv", index=False)
df_league.drop(['index', 'queue', 'name', 'leagueId', 'entries', 'rank'], axis=1).to_csv("../data/df_league_final.csv", index=False)

SyntaxError: invalid syntax (<ipython-input-46-2c05778d4ff7>, line 6)

>> ##### <a id='match_data_sub'> 2. 매치 데이터 </a> <font size=1> [목차](#index) </font>
>> - 먼저 Game ID를 받아온 후에
>>     - Game ID 당 10개인 것도 거의 없고, lane과 role도 잘못 적힌게 많음
>> - 실제 매치 데이터를 받아옴

In [45]:
# Game ID를 받아옴
df_match_info = pd.DataFrame()
for _idx, _account_id in tqdm(enumerate(df_league['accountId']), total=df_league.shape[0]):
    api_url = f"https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/{_account_id}?api_key={api_key}"
    r = requests.get(api_url)
    while r.status_code!=200: # 요청 제한 또는 오류로 인해 정상적으로 받아오지 않는 상태라면, 2초 간 시간을 지연
        time.sleep(2)
        r = requests.get(api_url)
    # 기존의 league 정보와 catesian product를 위해 임시 key 생성 후 merge
    df_league_temp = df_league.loc[_idx:_idx, :].copy()
    df_league_temp.loc[:, "key"] = 1
    df_match_info_temp = pd.DataFrame(r.json()['matches']).reset_index(drop=True)
    df_match_info_temp.loc[:, "key"] = 1
    df_match_info_temp = pd.merge(df_league_temp, df_match_info_temp, on="key")
    
    df_match_info = pd.concat([df_match_info, df_match_info_temp], axis=0)
df_match_info.to_csv("../data/df_match_info.csv", index=False)

df_match_info_final = df_match_info_final[[""]].copy()
df_match_info_final = df_match_info_final.rename(columns={"queue_x": "queue_value", "queue_y": "queue_key"})
df_match_info_final.to_csv("../data/df_match_info_final.csv", index=False)

SyntaxError: EOL while scanning string literal (<ipython-input-45-c21c3cbae8b1>, line 5)

In [15]:
# # 매치 데이터 받아옴
df_match = pd.DataFrame()
for _gaim_id in tqdm(df_match_info.gameId.unique(), total=df_match_info.shape[0]):
# for _game_id in tqdm(df_match_info.gameId.unique(), total=len(df_match_info.gameId.unique())):
    api_url = f"https://kr.api.riotgames.com/lol/match/v4/matches/{_game_id}?api_key={api_key}"
    r = requests.get(api_url)
    while r.status_code != 200:
        time.sleep(5)
        r = requests.get(api_url)
    r_json = r.json()
    df_match_temp = pd.DataFrame(list(r_json.values()), index=list(r_json.keys())).T
    df_match = pd.concat([df_match, df_match_temp], axis=0)
    
# DataFrame 내의 리스트들이 파일로 저장되었다가 불러지는 과정에서 문자로 인식됨
for column in ['teams', 'participants', 'participantIdentities']:
    df_match[column] = df_match[column].map(ast.literal_eval) # 각 값에 대해 eval 함수를 적용
# 버전 비교를 위해 파싱
df_match["gameVersion"] = df_match["gameVersion"].map(version.parse)


df_match.to_csv("../data/df_match.csv", index=False)

# 최신 버전과 CLASSIC 5x5 솔랭만 사용
df_match_final = df_match.query("gameVersion >= @version.parse('10.18') & gameMode == 'CLASSIC' & queueId == 420")
df_match_final.to_csv("../data/df_match_final.csv", index=False)

In [22]:
pd.DataFrame(df_match_final['teams'].iloc[1])

,teamId,win,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,towerKills,inhibitorKills,baronKills,dragonKills,vilemawKills,riftHeraldKills,dominionVictoryScore,bans
0,100,Win,True,False,False,False,False,False,11,2,1,2,0,0,0,"[{'championId': 7, 'pickTurn': 1}, {'championI..."
1,200,Fail,False,True,True,True,True,True,8,1,1,3,0,2,0,"[{'championId': -1, 'pickTurn': 6}, {'champion..."


In [25]:
## 매치 데이터에서 teams, participants, participantIdentities가 최종적으로 원하는 데이터이다.

df_team_matches = pd.DataFrame()
for i in trange(len(df_match_final)):
    temp_df = pd.DataFrame(df_match_final['teams'].iloc[i]) # teams 데이터를 2행 짜리 데이터프레임으로 변환
    temp_df['gameId'] = df_match_final['gameId'].iloc[i] # teams 데이터에 각 게임의 gameId 추가 (2행 마다 같은 값)
    # teams 데이터에 있는 bans 데이터를 5개의 변수로 추가한다
    ban_dict = {i: pd.DataFrame(temp_df['bans'][i]).iloc[:, 0] for i in range(2)} # 각 팀의 밴픽을 저장
    temp_ban = pd.DataFrame(ban_dict).T
    temp_ban.columns = [f'ban{i}' for i in range(1, 6)] # 열 이름 변경
    temp_df = pd.concat([temp_df, temp_ban], axis=1)

    df_team_matches = pd.concat([df_team_matches, temp_df])

df_team_matches.to_csv("../data/df_team_matches.csv", index=False)

100%|████████████████████████████████████████████████████████████████████████████| 22647/22647 [07:07<00:00, 52.92it/s]


In [43]:
# 각 타임라인에 찍힌 위치 정보가 필요한데, match-timelines 데이터에 모여있다.
# 그래서 이 데이터를 가져와야한다.

lst_match_timeline = []
for _game_id in tqdm(df_match_final['gameId'], total=df_match_final.shape[0]): # 각 게임 아이디마다 요청
    api_url = f'https://kr.api.riotgames.com/lol/match/v4/timelines/by-match/{_game_id}?api_key={api_key}'
    r = requests.get(api_url)
    while r.status_code!=200: # 요청 제한 또는 오류로 인해 정상적으로 받아오지 않는 상태라면, 3초 간 시간을 지연
        time.sleep(3)
        r = requests.get(api_url)
    temp_match = pd.DataFrame(list(r.json().values())[0]) # 전체 데이터 저장 (데이터 값에 딕셔너리 형태로 필요한 정보가 저장)
    temp_timeline = pd.DataFrame()
    len_timeline = temp_match.shape[0]
    for i in range(len_timeline): # 각 게임의 타임라인이 모두 다르기 때문 (1분 가량마다 타임라인이 찍힌다)
        temp_current_timeline = pd.DataFrame(temp_match['participantFrames'].iloc[i]).T
        if i != (len_timeline-1):
            temp_position = pd.DataFrame(list(temp_current_timeline['position'].values), index=temp_current_timeline.index)
            temp_current_timeline = pd.concat([temp_current_timeline, temp_position], axis=1)
            temp_current_timeline.drop('position', axis=1, inplace=True)
        temp_current_timeline['timestamp'] = temp_match['timestamp'][i]
        temp_timeline = pd.concat([temp_timeline, temp_current_timeline], axis=0, sort=False)
    lst_match_timeline.append(temp_timeline)

# f = open('MatchTimelineData.pickle', 'wb') # 리스트 안의 데이터프레임 형태이므로 바이너리 코드로 저장하기 위함임
# pickle.dump(match_timeline_list, f)
# f.close()

# 블랙리스트 되서 또 11788개의 데이터만 받아왔음
# match_df = match_df.iloc[:11788, :].copy()
# f = open('MatchTimelineData.pickle', 'rb')
# match_timeline_list = pickle.load(f)

In [48]:
with open("../data/lstT_match_timeline.pkl", "wb") as f:
    pickle.dump(lst_match_timeline, f)